一个小例子说明文本分类的基本流程，以及模型选择

## Data Preprocessing

In [1]:
import sys
sys.path.insert(0, '..')
import os

import numpy as np
import pandas as pd

In [2]:
import zipfile
with zipfile.ZipFile('./data/labeledTrainData.tsv.zip', 'r') as z:
    z.extractall('./data/')

In [3]:
data_train = pd.read_csv('./data/labeledTrainData.tsv', sep='\t')
print(data_train.shape)
data_train.head(5)

(25000, 3)


,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [4]:
data_train['review'][0]

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [5]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def clean_str(string):
    """
    Tokenization/string cleaning for all datasets 
    Every dataset is lower cased
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " \( ", string) 
    string = re.sub(r"\)", " \) ", string) 
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\s{2,}", " ", string)    
    return string.strip().lower()

stops = set(stopwords.words("english"))
def review_to_words(raw_review):
    review_text = BeautifulSoup(raw_review, "lxml").get_text()  
    words = clean_str(review_text)
    meaningful_words = [w for w in words.split(' ') if not w in stops]
    return(" ".join(meaningful_words))

In [6]:
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [7]:
texts = []
texts_with_stopwords = []
labels = []
for idx in range(data_train.review.shape[0]):
    text = review_to_words(data_train.review[idx])
    texts.append(text)
    texts_with_stopwords.append(clean_str(BeautifulSoup(data_train.review[idx], "lxml").get_text()))
    labels.append(data_train.sentiment[idx])

In [8]:
texts[0]

"stuff going moment mj 've started listening music , watching odd documentary , watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography , part feature film remember going see cinema originally released subtle messages mj 's feeling towards press also obvious message drugs bad m'kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts 20 minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans \\? nah , joe pesci 's character ranted wanted people know supplying drugs etc dunno , maybe hates mj 's music lots cool things like mj turning car robot whole speed demon sequence also , director must patience saint came filming kiddy bad sequ

In [9]:
texts_with_stopwords[0]

"with all this stuff going down at the moment with mj i 've started listening to his music , watching the odd documentary here and there , watched the wiz and watched moonwalker again maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent moonwalker is part biography , part feature film which i remember going to see at the cinema when it was originally released some of it has subtle messages about mj 's feeling towards the press and also the obvious message of drugs are bad m'kay visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him the actual feature film bit when it finally starts is only on for 20 m

## LogisticRegression

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression

text_clf = Pipeline([('vect', CountVectorizer(max_df=0.95, min_df=3)),
                     ('clf', LogisticRegression(C=3)),
])

parameters = {'vect__min_df': (1,3,5),
              'vect__ngram_range': [(1, 1), (1, 2)],
              'clf__C': (1,3,5,10),
}

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=4)
gs_clf = gs_clf.fit(texts, labels)
print(gs_clf.best_score_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

0.88664
clf__C: 1
vect__min_df: 1
vect__ngram_range: (1, 2)


In [21]:
text_clf = Pipeline([('vect', CountVectorizer(max_df=0.95, min_df=3, ngram_range=(1, 2))),
                     ('clf', LogisticRegression(C=3)),
])

parameters = {'vect__min_df': (1,3,5),
              'clf__C': (1,3,5,10),
}

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=4)
gs_clf = gs_clf.fit(texts_with_stopwords, labels)
print(gs_clf.best_score_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

0.89088
clf__C: 1
vect__min_df: 1


In [19]:
text_clf = Pipeline([('vect', TfidfVectorizer(max_df=0.95, min_df=3)),
                     ('clf', LogisticRegression(C=3)),
])

parameters = {'vect__min_df': (1,3,5),
              'vect__ngram_range': [(1, 1), (1, 2)],
              'clf__C': (1,3,5,10),
}

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=4)
gs_clf = gs_clf.fit(texts, labels)
print(gs_clf.best_score_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

0.89628
clf__C: 10
vect__min_df: 3
vect__ngram_range: (1, 2)


In [22]:
text_clf = Pipeline([('vect', TfidfVectorizer(max_df=0.95, min_df=3, ngram_range=(1, 2))),
                     ('clf', LogisticRegression(C=3)),
])

parameters = {'vect__min_df': (1,3,5),
              'clf__C': (1,3,5,10,15),
}

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=4)
gs_clf = gs_clf.fit(texts_with_stopwords, labels)
print(gs_clf.best_score_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

0.90036
clf__C: 15
vect__min_df: 3


futher tuning the parameters

In [18]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=3, ngram_range=(1, 2))
tfidf_features = tfidf_vectorizer.fit_transform(texts)
tfidf_features_with_stopwords = tfidf_vectorizer.fit_transform(texts_with_stopwords)

In [19]:
print(tfidf_features.shape)
print(tfidf_features_with_stopwords.shape)

(25000, 180058)
(25000, 265165)


In [24]:
lr = LogisticRegression()
parameters = {'C':[1,3,5,10,15], 'tol':[1e-6,1e-5,1e-4,1e-3]}
clf = GridSearchCV(lr, parameters, n_jobs=4)
clf.fit(tfidf_features, labels)

print(clf.best_score_)
print(clf.best_estimator_)

0.89772
LogisticRegression(C=15, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=1e-06,
          verbose=0, warm_start=False)


In [25]:
lr = LogisticRegression()
parameters = {'C':[1,3,5,10,15], 'tol':[1e-6,1e-5,1e-4,1e-3]}
clf = GridSearchCV(lr, parameters, n_jobs=4)
clf.fit(tfidf_features_with_stopwords, labels)

print(clf.best_score_)
print(clf.best_estimator_)

0.90124
LogisticRegression(C=15, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=1e-06,
          verbose=0, warm_start=False)


**最终，通过实验可知使用逻辑回归在IMDB数据集上的情感分类最优选择**：

- 使用2-grams的TF_IDF权重特征，min_df=3, tol=1e-06
- 语料预处理时保留停用词
- 最后交叉校验选择LR的最佳参数组合

## MultinomialNB

In [26]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

text_clf = Pipeline([('vect', CountVectorizer(max_df=0.95, min_df=3)),
                     ('clf', MultinomialNB(alpha=1)),
])

parameters = {'vect__min_df': (1,3,5),
              'vect__ngram_range': [(1, 1), (1, 2)],
              'clf__alpha': (0.01,0.1,1,3,5,10),
}

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=4)
gs_clf = gs_clf.fit(texts, labels)
print(gs_clf.best_score_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

0.87736
clf__alpha: 1
vect__min_df: 1
vect__ngram_range: (1, 2)


In [27]:
text_clf = Pipeline([('vect', CountVectorizer(max_df=0.95, min_df=1, ngram_range=(1, 2))),
                     ('clf', MultinomialNB(alpha=1)),
])

parameters = {'vect__min_df': (1,3,5),
              'clf__alpha': (0.01,0.1,1,3,5,10),
}

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=4)
gs_clf = gs_clf.fit(texts_with_stopwords, labels)
print(gs_clf.best_score_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

0.87808
clf__alpha: 1
vect__min_df: 1


In [28]:
text_clf = Pipeline([('vect', TfidfVectorizer(max_df=0.95, min_df=3, ngram_range=(1, 2))),
                     ('clf',  MultinomialNB(alpha=1)),
])

parameters = {'vect__min_df': (1,3,5),
              'clf__alpha': (0.01,0.1,1,3,5,10),
}

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=4)
gs_clf = gs_clf.fit(texts_with_stopwords, labels)
print(gs_clf.best_score_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

0.889
clf__alpha: 0.1
vect__min_df: 1


## RandomForestClassifier

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

text_clf = Pipeline([('vect', CountVectorizer(max_df=0.95, min_df=3)),
                     ('clf', RandomForestClassifier(n_estimators =250)),
])

parameters = {'vect__min_df': (1,3,5),
              'vect__ngram_range': [(1, 1), (1, 2)],
              'clf__n_estimators': (250, 500, 1000),
}

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=4)
gs_clf = gs_clf.fit(texts, labels)
print(gs_clf.best_score_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

0.86864
clf__n_estimators: 1000
vect__min_df: 1
vect__ngram_range: (1, 2)


In [11]:
text_clf = Pipeline([('vect', CountVectorizer(max_df=0.95, min_df=3, ngram_range=(1, 2))),
                     ('clf',  RandomForestClassifier(n_estimators =250)),
])

parameters = {'vect__min_df': (1,3,5),
              'clf__n_estimators': (500, 1000),
}

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=4)
gs_clf = gs_clf.fit(texts_with_stopwords, labels)
print(gs_clf.best_score_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

0.86768
clf__n_estimators: 1000
vect__min_df: 3


In [12]:
text_clf = Pipeline([('vect', TfidfVectorizer(max_df=0.95, min_df=3, ngram_range=(1, 2))),
                     ('clf',  RandomForestClassifier(n_estimators =250)),
])

parameters = {'vect__min_df': (1,3,5),
              'clf__n_estimators': (250, 500, 1000),
}

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=4)
gs_clf = gs_clf.fit(texts, labels)
print(gs_clf.best_score_)
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

0.86912
clf__n_estimators: 1000
vect__min_df: 1
